<h1 align=center><font size = 6>Airport Flight Delay Analysis</font></h1>

## Introduction

In this project, we will analyze the airline data set and gain insight into the airline industry. To do that, we will work with a Python visualization library, namely **plotly**. We will use the visualizions we create in **plotly** to create a web app using **flask** that will display the visualtions and insights gained from our exploration.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

1. [Datasets](#0)
2. [Downloading and Prepping Data](#2)<br>
3. [Data Exploration](#4) <br>
4. [Plotly](#6)<br>
4. [SQL](#8) <br>
5. [Web App](#10) <br>
</div>
<hr>

# Datasets<a id="0"></a>

Toolkits: This project heavily relies on [*pandas*](http://pandas.pydata.org/) and [**Numpy**](http://www.numpy.org/) for data wrangling, analysis, and visualization. The primary plotting library we will explore is [**Plotly**](https://plot.ly/).

Datasets: 

1. U.S Airline Delay and Cancellation Data from 2009 to 2018 - [Airline Delay Data](https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018/) from the Kaggle public data. The dataset is derived from the Department of Transportaions flight info of US domestic flights. Updated yearly, showing data for the entire years of 2009 to 2018. 


2. Airline Data - [Airline Data](https://www.kaggle.com/open-flights/airline-database) from the Kaggle public data website. The dataset contains a database of over 5000 airlines. The data presents airline name, IATA id, callsign, and country.


3. International IATA Airports Data - [Airport Data](https://rapidapi.com/easypnr/api/world-iata-airports) from Rapid API. The dataset consists of reliable airports list and details, with IATA codes. The Data is accessed through an API call. 

# Downloading and Prepping Data <a id="2"></a>

In [132]:
# Dependencies
import json
import pandas as pd
import matplotlib as plt
import numpy as np
from config import headers_airport

### Airport Data
<a class="anchor" id="airport_code"></a>

In [30]:
url = "https://airports-iata.p.rapidapi.com/airports"

response = requests.request("GET", url, headers=headers_airport)

airport_data = response.json()

airport_data

[{"locationName":"Anaa Airport","location":"Anaa, Tuamotus","country":"French Polynesia","iataCode":"AAA"},{"locationName":"Arrabury Airport","location":"Arrabury, Queensland","country":"Australia","iataCode":"AAB"},{"locationName":"El Arish International Airport","location":"El Arish","country":"Egypt","iataCode":"AAC"},{"locationName":"Ad-Dabbah Airport","location":"Al Dabbah","country":"Sudan","iataCode":"AAD"},{"locationName":"Rabah Bitat Airport (Les Salines Airport)","location":"Annaba","country":"Algeria","iataCode":"AAE"},{"locationName":"Apalachicola Municipal Airport","location":"Apalachicola, Florida","country":"United States","iataCode":"AAF"},{"locationName":"Arapoti Airport","location":"Arapoti, Paraná","country":"Brazil","iataCode":"AAG"},{"locationName":"Merzbrück Airport","location":"Aachen","country":"Germany","iataCode":"AAH"},{"locationName":"Arraias Airport","location":"Arraias, Tocantins","country":"Brazil","iataCode":"AAI"},{"locationName":"Cayana Airstrip","loca

In [37]:
# Load data into Pandas df
df_airport = pd.DataFrame(airport_data)

In [60]:
# Keep only the U.S. airports
df_USAirports = df_airport[df_airport['country'] == 'United States']
df_USAirports.head()

# Keep only the 'Airports'
df_USAirports = df_USAirports[df_USAirports['locationName'].apply(lambda x: 'Airport' in x)]

,locationName,location,country,iataCode
5,Apalachicola Municipal Airport,"Apalachicola, Florida",United States,AAF
15,Andrau Airpark,"Houston, Texas",United States,AAP
29,Lehigh Valley International Airport,"Allentown, Pennsylvania",United States,ABE
33,Abilene Regional Airport,"Abilene, Texas",United States,ABI
36,Ambler Airport,"Ambler, Alaska",United States,ABL


In [78]:
df_USAirports[df_USAirports['location'] == 'Los Angeles, California']

,locationName,location,country,iataCode
4048,Los Angeles International Airport,"Los Angeles, California",United States,LAX
7030,Whiteman Airport,"Los Angeles, California",United States,WHP


In [80]:
df_USAirports

,locationName,location,country,iataCode
5,Apalachicola Municipal Airport,"Apalachicola, Florida",United States,AAF
29,Lehigh Valley International Airport,"Allentown, Pennsylvania",United States,ABE
33,Abilene Regional Airport,"Abilene, Texas",United States,ABI
36,Ambler Airport,"Ambler, Alaska",United States,ABL
42,Aberdeen Regional Airport,"Aberdeen, South Dakota",United States,ABR
49,Southwest Georgia Regional Airport,"Albany, Georgia",United States,ABY
52,Antrim County Airport,"Bellaire, Michigan",United States,ACB
59,Nantucket Memorial Airport,"Nantucket, Massachusetts",United States,ACK
65,Waco Regional Airport,"Waco, Texas",United States,ACT
67,Arcata-Eureka Airport,"McKinleyville, California",United States,ACV


### Airline Code Data

In [125]:
# read airlines data into pandas data frame
df_airline = pd.read_csv('data/airlines.csv')
df_airline.head()

,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N


In [100]:
# Delete/rename columns
df_airline = df_airline.drop(['Airline ID', 'Alias'], axis=1)

,Name,IATA,ICAO,Callsign,Country,Active
0,Unknown,-,NaN,\N,\N,Y
1,Private flight,-,NaN,NaN,NaN,Y
2,135 Airways,NaN,GNL,GENERAL,United States,N
3,1Time Airline,1T,RNX,NEXTIME,South Africa,Y
4,2 Sqn No 1 Elementary Flying Training School,NaN,WYT,NaN,United Kingdom,N


### Airport Delay Data

In [128]:
# Read the Airport data from 2015 - 2018 into a pandas dataframe then combine into one df
df_2018 = pd.read_csv('data/2018.csv')
df_2017 = pd.read_csv('data/2017.csv')
df_2016 = pd.read_csv('data/2016.csv')
df_2015 = pd.read_csv('data/2015.csv')

df_delay = pd.concat([df_2018, df_2017, df_2016, df_2015])

df_delay.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
# Determine dimensions of df
df_delay.shape

(24324804, 28)

In [129]:
# Drop and rename columns in the df
df_delay = df_delay.drop(['CANCELLATION_CODE', 'Unnamed: 27'], axis=1)

df_delay = df_delay.rename(columns={"FL_DATE":"FLIGHT_DATE",
                                    "OP_CARRIER":"AIRLINE",
                                    "OP_CARRIER_FL_NUM":"FLIGHT_NUMBER",
                                    "CRS_DEP_TIME": "SCHEDULED_DEP", 
                                    "CRS_ARR_TIME":"SCHEDULED_ARR", 
                                    "CRS_ELAPSED_TIME":"SCHEDULED_ELAPSED_TIME",
                                   })

# Repalce Nan values with 0
df_delay['CARRIER_DELAY'] = df_delay['CARRIER_DELAY'].replace(np.nan, 0)
df_delay['WEATHER_DELAY'] = df_delay['WEATHER_DELAY'].replace(np.nan, 0)
df_delay['NAS_DELAY'] = df_delay['NAS_DELAY'].replace(np.nan, 0)
df_delay['SECURITY_DELAY'] = df_delay['SECURITY_DELAY'].replace(np.nan, 0)
df_delay['LATE_AIRCRAFT_DELAY'] = df_delay['LATE_AIRCRAFT_DELAY'].replace(np.nan, 0)

In [130]:
# Display the data
df_delay.head(15)

,FLIGHT_DATE,AIRLINE,FLIGHT_NUMBER,ORIGIN,DEST,SCHEDULED_DEP,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,DIVERTED,SCHEDULED_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,...,0.0,268.0,250.0,225.0,1605.0,0.0,0.0,0.0,0.0,0.0
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,...,0.0,99.0,83.0,65.0,414.0,0.0,0.0,0.0,0.0,0.0
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,...,0.0,134.0,126.0,106.0,846.0,0.0,0.0,0.0,0.0,0.0
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,...,0.0,190.0,182.0,157.0,1120.0,0.0,0.0,0.0,0.0,0.0
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,...,0.0,112.0,106.0,83.0,723.0,0.0,0.0,0.0,0.0,0.0
5,2018-01-01,UA,2422,ORD,OMA,2241,2244.0,3.0,15.0,2259.0,...,0.0,93.0,79.0,62.0,416.0,0.0,0.0,0.0,0.0,0.0
6,2018-01-01,UA,2421,IAH,LAS,750,747.0,-3.0,14.0,801.0,...,0.0,206.0,193.0,173.0,1222.0,0.0,0.0,0.0,0.0,0.0
7,2018-01-01,UA,2420,DEN,CID,1324,1318.0,-6.0,11.0,1329.0,...,0.0,115.0,102.0,85.0,692.0,0.0,0.0,0.0,0.0,0.0
8,2018-01-01,UA,2419,SMF,EWR,2224,2237.0,13.0,10.0,2247.0,...,0.0,314.0,299.0,280.0,2500.0,0.0,0.0,0.0,0.0,0.0
9,2018-01-01,UA,2418,RIC,DEN,1601,1559.0,-2.0,12.0,1611.0,...,0.0,252.0,237.0,217.0,1482.0,0.0,0.0,0.0,0.0,0.0


# Data Exploration <a id="4"></a>

# Plotly <a id="6"></a>

# SQL <a id="8"></a>

# Web App <a id="10"></a>